Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets

- [ ] Continue to clean and explore your data. 
- [ ] For the evaluation metric you chose, what score would you get just by guessing?
- [ ] Can you make a fast, first model that beats guessing?

**We recommend that you use your portfolio project dataset for all assignments this sprint.**

**But if you aren't ready yet, or you want more practice, then use the New York City property sales dataset for today's assignment.** Follow the instructions below, to just keep a subset for the Tribeca neighborhood, and remove outliers or dirty data. [Here's a video walkthrough](https://youtu.be/pPWFw8UtBVg?t=584) you can refer to if you get stuck or want hints!

- Data Source: [NYC OpenData: NYC Citywide Rolling Calendar Sales](https://data.cityofnewyork.us/dataset/NYC-Citywide-Rolling-Calendar-Sales/usep-8jbt)
- Glossary: [NYC Department of Finance: Rolling Sales Data](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page)

# Original assignment cells:

I will be continuing the exploration and wrangle function on cells beneath them, including questions and actions from the assignment cells where applicable. 

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*
    !pip install pandas-profiling==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [ ]:
# Read New York City property sales data
import pandas as pd
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

Your code starts here:

In [ ]:
# Change column names: replace spaces with underscores


In [ ]:
# Get Pandas Profiling Report


In [ ]:
# Keep just the subset of data for the Tribeca neighborhood
# Check how many rows you have now. (Should go down from > 20k rows to 146)


In [ ]:
# Q. What's the date range of these property sales in Tribeca?


In [ ]:
# The Pandas Profiling Report showed that SALE_PRICE was read as strings
# Convert it to integers


In [ ]:
# Q. What is the maximum SALE_PRICE in this dataset?


In [ ]:
# Look at the row with the max SALE_PRICE


In [ ]:
# Get value counts of TOTAL_UNITS
# Q. How many property sales were for multiple units?


In [ ]:
# Keep only the single units


In [ ]:
# Q. Now what is the max sales price? How many square feet does it have?


In [ ]:
# Q. How often did $0 sales occur in this subset of the data?

# There's a glossary here: 
# https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

# It says:
# A $0 sale indicates that there was a transfer of ownership without a 
# cash consideration. There can be a number of reasons for a $0 sale including 
# transfers of ownership from parents to children. 



In [ ]:
# Look at property sales for > 5,000 square feet
# Q. What is the highest square footage you see?


In [ ]:
# What are the building class categories?
# How frequently does each occur?


In [ ]:
# Keep subset of rows:
# Sale price more than $0, 
# Building class category = Condos - Elevator Apartments

# Check how many rows you have now. (Should be 106 rows.)


In [ ]:
# Make a Plotly Express scatter plot of GROSS_SQUARE_FEET vs SALE_PRICE


In [ ]:
# Add an OLS (Ordinary Least Squares) trendline,
# to see how the outliers influence the "line of best fit"


In [ ]:
# Look at sales for more than $35 million

# All are at 70 Vestry Street
# All but one have the same SALE_PRICE & SALE_DATE
# Was the SALE_PRICE for each? Or in total?
# Is this dirty data?


In [ ]:
# Make a judgment call:
# Keep rows where sale price was < $35 million

# Check how many rows you have now. (Should be down to 90 rows.)


In [ ]:
# Now that you've removed outliers,
# Look again at a scatter plot with OLS (Ordinary Least Squares) trendline


In [ ]:
# Select these columns, then write to a csv file named tribeca.csv. Don't include the index.


# My Data Below:

Below I will continue yesterday's assignment on cleaning and organizing my data, and fixing a basic model.

Questions/Cells from above to include:
- Change column names: replace spaces with underscores
- Get Pandas Profiling Report


Note: The majority of my EDA can be found in the previous notebook.  Today's EDA will be a continuation of yesterday's work in this notebook.

## Imports

In [ ]:
!pip install eli5

     |████████████████████████████████| 112kB 3.4MB/s 


In [ ]:
#Imports
%matplotlib inline

import pandas as pd
import numpy as np
import plotly.express as px
import eli5

from sklearn.linear_model import LinearRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from eli5.sklearn import PermutationImportance

import matplotlib.pyplot as plt


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.

Using TensorFlow backend.


## EDA

Importing our data and basic EDA

In [ ]:
#Import our data

#URL to our data on my github repo for build week
url = 'https://raw.githubusercontent.com/JeremySpradlin/DS-Unit-2-Build-Week/master/sunspot_data.csv'


df = pd.read_csv(url)
df.head()

,Unnamed: 0,Year,Month,Day,Date In Fraction Of Year,Number of Sunspots,Standard Deviation,Observations,Indicator
0,0,1818,1,1,1818.001,-1,-1.0,0,1
1,1,1818,1,2,1818.004,-1,-1.0,0,1
2,2,1818,1,3,1818.007,-1,-1.0,0,1
3,3,1818,1,4,1818.010,-1,-1.0,0,1
4,4,1818,1,5,1818.012,-1,-1.0,0,1


Our target that we will be predicting will be the number of sun spots on any particualr date, shown in the column `Number of Sunspots`

In [ ]:
px.scatter(df, x='Date In Fraction Of Year', y='Number of Sunspots', color='Observations')

## Wrangle Function

Below we will define our wrangle function that will be used to clean our dataset and prepare for fitting. It needs to perform the following actions:

- Remove spaces from column names
- Remove upper case letters from column names
- `Number of Sunspots`
 - Remove all -1's, replace with NAN
 - Verify that 0's are accompanied with verifcation with observations
    - Handled by removing rows with observations=0
- Remove Columns:
 - `Indicator`
 - `Unnamed: 0`
-Remove Rows:
 - Where `Observations` = 0
 - Where `Sunspots` = NAN
- Split our data into training and validation sets
 - Since we are looking at data over a 200 yr period, we will split the data chronologically.
   - Training Set: 1802 - 1902
   - Validation Set: 1903 - 1953
   - Testing Set: 1954 - 2018

**NOTE:** Testing set sizes might change in the future



In [ ]:
#Create our data wrangling function

def wrangle(df):
  """This function will take in a dataframe of Sunspot activity
  and perform different functions and actions on it to 
  prepare the dataset for training in a predictive model."""

  #Remove spaces from column names and change to lowercase
  df.columns = df.columns.str.lower().str.replace(' ', '_')

  #Replace -1's in target column
  df['number_of_sunspots'].replace(-1, np.NaN, inplace=True)

  #Remove columns
  df = df.drop(['indicator', 'unnamed:_0'], axis=1)

  #Remove observations with missing values or no observations
  df = df.dropna()
  mask = df[(df['observations'] == 0)].index
  df = df.drop(mask)

  #Split our dataset into data and target sets
  y = df['number_of_sunspots']
  X = df.drop('number_of_sunspots', axis=1)

  #Create Training set
  X_train = X[(X['year'] <= 1902)]
  y_train = y[y.index.isin(X_train.index)]

  #Create validation set
  X_val = X[(X['year'] > 1902) & (X['year'] <= 1952)]
  y_val = y[y.index.isin(X_val.index)]

  #Create test set
  X_test = X[(X['year'] > 1952)]
  y_test = y[y.index.isin(X_test.index)]


  #Return the dataframes
  return X_train, y_train, X_val, y_val, X_test, y_test


In [ ]:
X_train, y_train, X_val, y_val, X_test, y_test = wrangle(df)
X_train.shape, y_train.shape

((27798, 6), (27798,))

### Baseline

In [ ]:
guess = y_train.mean()
errors = guess - y_train
mae = errors.abs().mean()
print('Our naive baseline mae is:', mae)

Our naive baseline mae is: 57.92122350081531


### Create our Pipeline

In [ ]:
model = make_pipeline(
    SimpleImputer(),
    StandardScaler(),
    LinearRegression()
)

In [ ]:
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('simpleimputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearregression',
                 LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
                                  normalize=False))],
         verbose=False)

In [ ]:
#Check our accuracy on our different sets
print('Training Accuracy: ', model.score(X_train, y_train))
print('Validation Accuracy: ', model.score(X_val, y_val))

Training Accuracy:  0.9062704166214505
Validation Accuracy:  0.7675162495780349


In [ ]:
print('Testing Accuracy: ', model.score(X_test, y_test))

Testing Accuracy:  0.6303832707461011
